In [ ]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage langchain-google-vertexai google-cloud-bigquery

In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

## Initial Vertex AI

In [ ]:
# init the aiplatform package
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Helper Functions to answer questions using Gemini and PaLM LLMS


In [ ]:
from google.cloud.aiplatform_v1beta1.types.content import SafetySetting, HarmCategory
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.language_models import TextGenerationModel

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Creaning the Data")
    return "An Error Ocuured Creaning the Data"


def answer_question_palm(prompt):
  parameters = {
        "candidate_count": 1,
        "max_output_tokens": 1024,
        "temperature": 0.9,
        "top_p": 1
    }

  model = TextGenerationModel.from_pretrained("text-bison")
  response = model.predict(
        prompt,
        **parameters
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Creaning the Data")
    return "An Error Ocuured Creaning the Data"

## Function to Search BigQuery and Return Content

In [ ]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.bbc_news_with_embeddings`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )


  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"
    #print("Title: {0} - Body: {1}".format(row.title, row. body[:50]))
  return data

In [ ]:
def build_prompt(data, question):
    prompt = """
    context: Answer the question using the following Information.

    Information: {0}

    question: {1}

    """.format(data, question)
    return prompt

In [ ]:
def answer_question(question):

  data = run_search(question)
  prompt = build_prompt(data, question)

  answer_gemini = answer_question_gemini(prompt)
  answer_palm = answer_question_palm(prompt)

  return answer_gemini, answer_palm

In [ ]:
from IPython.core.display import display, HTML


QUESTION = "Tell me about the US Economy"
#QUESTION = "What is Microsoft working on"
#QUESTION = "What is Google in the news for?"
#QUESTION = "Has there been any political unrest on Africa?"

answer_gemini, answer_palm = answer_question(QUESTION)

print(QUESTION)
print("--------------------------------")
display("Gemini answer: {0}".format(answer_gemini))
print("--------------------------------")
display("PaLM answer: {0}".format(answer_palm))

